In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import scanpy.api as sc
from scanpy.neighbors import Neighbors
import scanorama

# Evaluating Alignment Score For Batch Correction

This notebook evaluates an "alignment score" to quantify the effects of batch correction. This notebook uses this metric to measure the performance of the Scanorama algorithm on two batches of Olivier's CAOV3 data with one batch receiving an addition of random gaussian noise.

# Load Data

In [ ]:
B1 = sc.read_10x_mtx("../data/Harismendy_data/170206/CAOV3", var_names="gene_symbols")

In [ ]:
B2 = sc.read_10x_mtx("../data/Harismendy_data/170315/CAOV3", var_names="gene_symbols")

## Downsample For Performance

In [ ]:
def downsample_anndata(ad, n):
    rand_obs = np.random.choice(list(ad.obs.index), size=n, replace=False)
    return ad[rand_obs,:]

In [ ]:
CAOV3_B1 = downsample_anndata(B1, 500)
CAOV3_B2 = downsample_anndata(B2, 500)

In [ ]:
CAOV3_B1.shape

In [ ]:
CAOV3_B2.shape

# Implement Alignment Score

In [ ]:
INPUT_ANNS = [CAOV3_B1, CAOV3_B2]

In [ ]:
min_cells = min(list(map(lambda x: x.n_obs, INPUT_ANNS)))
DS_ANNS = list(map(lambda x: downsample_anndata(x, min_cells), INPUT_ANNS))

## Merge Experiments

In [ ]:
COMB_ANN = DS_ANNS[0].concatenate(DS_ANNS[1:], index_unique="_")
COMB_ANN.shape

## Find Closest 1% Neighbors For All Cells

In [ ]:
n_batch = len(INPUT_ANNS)

In [ ]:
n_neighbors = max(int(0.01*min_cells), 10)
n_neighbors

In [ ]:
sc.pp.neighbors(COMB_ANN, n_neighbors=22, knn=True)

## Calculate Alignment Score

In [ ]:
rows, cols = COMB_ANN.uns['neighbors']['distances'].nonzero()

In [ ]:
def get_batch(i, adata):
    """
    Returns the batch number of a cell at index i in adata
    """
    return int(adata.obs.index[i].split('_')[1])

In [ ]:
neighbor_counts = {}  #{cell_id: #same_batch}

for x, y in zip(rows, cols):
    try:
        neighbor_counts[x]
    except KeyError:
        neighbor_counts[x] = 0
    cell_batch = get_batch(x, COMB_ANN)
    neigh_batch = get_batch(y, COMB_ANN)
    if cell_batch == neigh_batch:
        neighbor_counts[x] += 1

In [ ]:
(np.mean(list(neighbor_counts.values()))/n_neighbors)*n_batch

# Summary Implementation Function

This cell defines a single function to run the above analysis given a list of AnnData

In [ ]:
def downsample_anndata(ad, n):
    rand_obs = np.random.choice(list(ad.obs.index), size=n, replace=False)
    return ad[rand_obs,:]

def get_batch(i, adata):
    """
    Returns the batch number of a cell at index i in adata
    """
    return int(adata.obs.index[i].split('_')[1])

def score_batch_corr(anns, n_neighbors):
    min_cells = min(list(map(lambda x: x.n_obs, anns)))
    ds_anns = list(map(lambda x: downsample_anndata(x, min_cells), anns))
    comb_ann = ds_anns[0].concatenate(ds_anns[1:], index_unique="_")
    sc.tl.pca(comb_ann)
    sc.pp.neighbors(comb_ann, n_neighbors=n_neighbors, knn=True)
    rows, cols = comb_ann.uns['neighbors']['distances'].nonzero()
    neighbor_counts = {}  #{cell_id: #same_batch}
    for x, y in zip(rows, cols):
        try:
            neighbor_counts[x]
        except KeyError:
            neighbor_counts[x] = 0
        cell_batch = get_batch(x, comb_ann)
        neigh_batch = get_batch(y, comb_ann)
        if cell_batch != neigh_batch:
            neighbor_counts[x] += 1
    sc.pl.pca(comb_ann, color='batch', show=True, save=False)
    return (np.mean(list(neighbor_counts.values()))/n_neighbors)*len(anns)

In [ ]:
N_NEIGHBORS = int(0.01*CAOV3_B1.shape[0])
N_NEIGHBORS = 10

In [ ]:
score_batch_corr([CAOV3_B1, CAOV3_B2], N_NEIGHBORS)

# Add Random Gaussian Noise To Olivier's Data

In [ ]:
CAOV3_B2_GAUSS = CAOV3_B2.copy()
rows, cols = CAOV3_B2_GAUSS.X.nonzero()
count = 0
for x, y in zip(rows, cols):
    #if np.random.rand() < 0.8:
    #    count += 1
    #    continue
    CAOV3_B2_GAUSS.X[x, y] += np.random.normal(loc=50, scale=100)
    count += 1
    if count%100000 == 0:
        print(float(count)/len(rows)*100)
#del CAOV3_B2

In [ ]:
score_batch_corr([CAOV3_B1, CAOV3_B2_GAUSS], N_NEIGHBORS)

# Correct Data With Scanorama

In [ ]:
CAOV3_B1_CORR, CAOV3_B2_GAUSS_CORR = scanorama.correct_scanpy([CAOV3_B1, CAOV3_B2_GAUSS])
score_batch_corr([CAOV3_B1_CORR, CAOV3_B2_GAUSS_CORR], N_NEIGHBORS)